In [1]:
#M.G. 5/23/17, Most recent update 8/14/17. This is likely the finished product, barring any enhancements made to run-time
#efficency, or generality or functionality of code. No solution has been found as of 8/14/17. The associated papers are 
#'The Reinhardt Conjecture as an Optimal Control Problem' and 'Reinhardt Project Summary', authors Thomas Hales and
#Matthew Gerstbrein, respectively.

t = var('t') #Global time parameter
[Ze, On, Tw] = [0, 1, 2] #Variable substitutions of integers 0, 1, and 2
#Dummy variables, utilized in code for substitution
dx_init, dy_init, dx_fin, dy_fin, dm, dc0, dc1, dt, dtf = var('dx_init dy_init dx_fin dy_fin dm dc0 dc1 dt dtf')

s = exp(dc1*dt) #s is defined as in section 4.1 of [Hal17]
star = var('star') #Intended as top right entry of g(s) as defined in [Hal17] section 4.1
g_temp = matrix.identity(2) + ((s-1)/(dc1^(2)*dc0*s))*matrix([[dc0 - dm, star], [1, dm - dc0*s]]) #Matrix g(s); see section 4.1 
entry01 = solve(det(g_temp) == 1, star)[0].right().full_simplify() #Value of missing entry of g(s)
gs = g_temp.substitute(star == entry01)
R = matrix([[N(cos(pi/3)), N(-sin(pi/3))], [N(sin(pi/3)), N(cos(pi/3))]]) #R as defined in [Hal17] section 3.5
R_inverse = R.inverse() # = matrix([[N(cos(-pi/3)), N(-sin(-pi/3))], [N(sin(-pi/3)), N(cos(-pi/3))]])
J = matrix([[0, -1], [1, 0]]) #J as defined in [Hal17] section 2.2
unit_circle = parametric_plot((lambda t: cos(t), lambda t: sin(t)), (t, 0, 2*pi), color = 'red') #See 'plot_comparison' below

def mobius(M, z):
    [[a, b], [c, d]] = M
    return N((a*z + b)/(c*z + d))

def linfrac(z): #Mobius transformation R.z, with R as defined in [Hal17] section 3.5
    return mobius(R, z)
    
def inverse_linfrac(z):
    return mobius(R_inverse, z)
    
def c0c1(z_init, m): 
    c0 = N(z_init.real() + m)
    c1 = N(z_init.imag()/c0)
    return (c0, c1)

def z_hat(z):
    return matrix([[z.imag(), z.real()], [0, 1]])

def X(z):
    return z_hat(z)*J*z_hat(z).inverse()

def e(i, z_init, t): #Inputs i = 0, 1, 2. IMPORTANT: e(i = 0) (notation e0) corresponds to e3 in [Hal17]!!
    if i != 1:
        if i == 0:
            m = N(1/sqrt(3))
        else:
            m = N(-1/sqrt(3))
        (c0, c1) = c0c1(z_init, m)
        (x, y) = (-m + c0*exp(c1*t), c0*c1*exp(c1*t))
        return N(x + y*I)
    else: #Utilize linear fractional transformation; true e1 parametrization is quite complex
        Rz_init = linfrac(z_init)
        Rz_t = e(0, Rz_init, t)
        return N(inverse_linfrac(Rz_t))

#solve(dx_fin == -dm + (dx_init + dm)*e^((dy_init/(dx_init + dm))*t), t)[0].right().full_simplify()
def compute_time(dm, dx_init, dx_fin, dy_init):
    solve_output = (dm + dx_init)*log((dm + dx_fin)/(dm + dx_init))/dy_init
    return solve_output

def get_time(z_init, z_fin): #Only for use for two points on an e2 or e3 trajectory, NOT an e1 trajectory
    if z_init.imag() <= z_fin.imag(): #if-else required to determine the sign of m
        m = N(1/sqrt(3)) 
    else:
        m = N(-1/sqrt(3))
    return compute_time(m, z_init.real(), z_fin.real(), z_init.imag())
    
#The input here should be the terminal x-point you choose after initially traveling along e3; everything is determined from here
def simple_control_trajectory(x1):
    (x0, y0) = (0, 1)
    z0 = x0 + y0*I
    y1 = N(sqrt(3)*x1 + 1)
    z1 = x1 + y1*I
    t1 = get_time(z0, z1)
    z2 = -x1 + y1*I  #By symmetry (across imaginary axis)
    t2 = get_time(linfrac(z1), linfrac(z2))
    t3 = get_time(z2, z0) #Note that t3 == t1
    return [[Ze, N(z0), N(t1)], [On, N(z1), N(t2)], [Tw, N(z2), N(t3)]] #Zero-th entry in each of the 3 3-tuples is the 'i' input for function e 

def control_trajectory(*args): #The general control trajectory, accepting 1, 2, or 3 inputs.
    if len(args) == 1:
        return simple_control_trajectory(*args)
    elif len(args) == 2: #For 2-param, be sure that x1_p is (strictly) between -sqrt(1/12) and sqrt(1/12), else line exits star region
        (x1, x1_p) = args
        [[Ze, z0, t1], [On, z1, t2], [Tw, z2, t3]] = simple_control_trajectory(x1)
        y1_p = N(-sqrt(3)*x1_p + 1); z1_p = x1_p + y1_p*I
        t1_p = get_time(z0, z1_p)
        z2_p = -x1_p + y1_p*I
        t2_p = get_time(linfrac(z1_p), linfrac(z2_p))
        t3_p = get_time(z2_p, z0)
        return [[Ze, N(z0), N(t1)], [On, N(z1), N(t2)], [Tw, N(z2), N(t3 + t1_p)], [On, N(z1_p), N(t2_p)], [Ze, N(z2_p), N(t3_p)]]
    elif len(args) == 3:
        (x1, x1_p, x1_p2) = args
        [[Ze, z0, t1], [On, z1, t2], [Tw, z2, t3]] = simple_control_trajectory(x1)
        [[Ze, z0_temp, t1_p], [On, z1_temp, t2_p], [Tw, z2_temp, t3_p]] = simple_control_trajectory(x1_p)
        [[Ze, z0_temp2, t1_p2], [On, z1_temp2, t2_p2], [Tw, z2_temp2, t3_p2]] = simple_control_trajectory(x1_p2)
        [z1_p, z2_p, z0_p] = [linfrac(z1_temp), linfrac(z2_temp), linfrac(z0_temp)]
        [z1_p2, z2_p2, z0_p2] = [inverse_linfrac(z1_temp2), inverse_linfrac(z2_temp2), inverse_linfrac(z0_temp2)]
        return [[Ze, z0, t1], [On, z1, t2], [Tw, z2, t3 + t1_p], [Ze, z1_p, t2_p], [On, z2_p, t3_p + t1_p2], [Tw, z1_p2, t2_p2], [Ze, z2_p2, t3_p2]]
    else:
        print("Enter in an appropriate number of parameters; either 1, 2, or 3. Or, edit the function however you'd like!")
        
def path(c, t):
    if c == []:
        return 0 + 1*I
    elif t <= c[0][2]:
        return e(c[0][0], c[0][1], t)
    else:
        return path(c[1:], t - c[0][2])
    
def tf(control):
    T = 0
    for i in range(0, len(control)):
        T = T + control[i][2]
    return T
        
def bar(c): #c = control_trajectory(*args)
    arr = [c[0][1]]
    for i in range(0, len(c)-1):
        arr.append(mobius(R^(c[i][0]-c[i+1][0]), e(0, arr[i], c[i][2])))
    return arr
    
#(3/2)*integral((x^(2) + y^(2) + 1)/y, t, 0, tf)    
def IF(z, tf): #IF abbreviation for 'integral formula'
    m = N(1/sqrt(3))
    (c0, c1) = c0c1(z, m)
    x = -m + c0*exp(c1*t); y = c0*c1*exp(c1*t)  #Solutions to differential equations; see (34) in [Hal17]
    integral_output = 3/2*-(m^2 - (c0^2*c1^2 + c0^2)*exp(2*c1*tf) + (c0^2*c1^2 + 2*c0*c1*m*tf + c0^2 - m^2 - 1)*exp(c1*tf) + 1)*exp(-c1*tf)/(c0*c1^2)
    #To clarify, the factor of 3/2 that appears in front of integral_output is the same 3/2 that is in
    #front of the integral itself, as given above this function, and also by (18)
    return integral_output

def cost(*args):
    c = control_trajectory(*args); b = bar(c)
    total_cost = 0
    for i in range(0, len(c)):
        total_cost += IF(b[i], c[i][2])
    return total_cost
    

#Plots not reliable for large values of tf, e.g. tf == 10; likely that parametric_plot becomes less refined (it plots over further spaced out points and approximates in between)
def plot_path(*args):
    control = control_trajectory(*args)
    t_f = tf(control)
    return parametric_plot((lambda t: path(control, t).real(), lambda t: path(control, t).imag()), (t, 0, t_f))


In [1]:
#gs.substitute([dt == t, dc0 == c0, dc1 == c1, dm == N(1/sqrt(3))]) is  equal to matrix([[(c0 - m)*(e^(c1*t) - 1)*e^(-c1*t)/(c0*c1^2) + 1, (c0*m - m^2 - (c0^2*c1^2 + c0^2 - c0*m)*e^(c1*t))*(e^(c1*t) - 1)*e^(-c1*t)/(c0*c1^2)], [(e^(c1*t) - 1)*e^(-c1*t)/(c0*c1^2), -(c0*e^(c1*t) - m)*(e^(c1*t) - 1)*e^(-c1*t)/(c0*c1^2) + 1]])
#where m = N(1/sqrt(3))
def gamma(i, z, t):
    (c0, c1) = c0c1(z, N(1/sqrt(3)))
    gamma0 = gs.substitute([dt == t, dc0 == c0, dc1 == c1, dm == N(1/sqrt(3))])
    return (R^i)*gamma0*(R^(-i))

def g(c, t): #c = control_trajectory(*args)
    b = bar(c); T = c[0][2]; M = identity_matrix(2); i = 0
    while t > T:
        M *= gamma(c[i][0], b[i], c[i][2])
        T += c[i+1][2]
        i += 1
    M *= gamma(c[i][0], b[i], t - (T - c[i][2]))
    return M
    
def e_star(j): #Inputs are j = 0, 1, 2, 3, 4, 5
    return vector([N(cos(2*pi*j/6)), N(sin(2*pi*j/6))])

def sigma(j, control, t):
    return g(control, t)*e_star(j)

def plot_g(i1, j1, i2, j2, *args): 
    #Not a necessary function; use if you would like to verify that (g_i1j1(t), g_i2j2(t)) is a differentiable graph (for any pair of indices)
    control = control_trajectory(*args)
    t_f = tf(control)
    return parametric_plot((lambda t: g(control, t)[i1][j1], lambda t: g(control, t)[i2][j2]), (t, 0, t_f))

def plot_sigma(j, control): #control = control_trajectory(*args)
    t_f = tf(control)
    return parametric_plot((lambda t: sigma(j, control, t)[0], lambda t: sigma(j, control, t)[1]), (t, 0, t_f))

def plot_shape(*args):
    c = control_trajectory(*args)
    P = point((0, 0))
    for j in range(0, 6):
        P = P + plot_sigma(j, c)
    return P

def plot_comparison(*args):
    return plot_shape(*args) + unit_circle

def error(*args):
    control = control_trajectory(*args)
    t_f = tf(control)
    [[a, b], [c, d]] = g(control, t_f)
    return (a - d)^(2) + (b + c)^(2)

def error_search(*args): #Arbitrary epsilon-tolerance is chosen; it can be changed. 0.25 is chosen so as to avoid numerical instability towards edges of star-region(it can also be changed).
    if len(args) == 1:
        [n1] = args
        for i in range(0, n1):
            if error(0.25*i/n1) < 0.0000001:
                print([i, 0.25*i/n1, error(0.25*i/n1)])
    elif len(args) == 2:
        [n1, n2] = args
        for i in range(0, n1):
            for j in range(0, n2):
                if error(0.25*i/n1, 0.25*j/n2) < 0.0000001:
                    print([[i, j], [0.25*i/n1, 0.25*j/n2], error(0.25*i/n1, 0.25*j/n2)])
    elif len(args) == 3:
        [n1, n2, n3] = args
        for i in range(0, n1):
            for j in range(0, n2):
                for k in range(0, n3):
                    if error(0.25*i/n1, 0.25*j/n2, 0.25*k/n3) < 0.0000001:
                        print([[i, j, k], [0.25*i/n1, 0.25*j/n2, 0.25*k/n3], error(0.25*i/n1, 0.25*j/n2, 0.25*k/n3)])
    else:
        print('Appropriate number of parameters is 1, 2, or 3.')
    print('Done')
